# Lab 7: 提示词工程(Prompt Engineering 101)

# 1.chatGPT是如何实现的

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()  # 敏感信息不能明文展示
api_key = os.getenv('OPENAI_API_KEY')

In [3]:
from openai import OpenAI

client = OpenAI(
            api_key=api_key,
            base_url="https://api.aaaapi.com/v1"
        )
# 创建一个简单的聊天完成请求
user_prompt = "你好,请介绍一下你自己"

response = client.chat.completions.create(
    model="gpt-4o",  
    messages=[
        {"role": "user", "content": user_prompt}
    ]
)
# 返回助手的回复
assistant_reply = response.choices[0].message.content


# 测试代码
print(f"用户: {user_prompt}")
print(f"AI助手: {assistant_reply}")


用户: 你好,请介绍一下你自己
AI助手: 你好！我是一个大型语言模型助手，由OpenAI开发，旨在帮助用户获取信息、解答问题和提供建议。我可以协助完成多种任务，包括提供教育支持、技术帮助、创意写作等等。如果你有任何问题或者需要帮助，请随时告诉我！


增加系统提示词

In [3]:
system_prompt = "你是一个谜语人，总是用谜语来对话。谜底必须对应用户的答案"
response = client.chat.completions.create(
    model="gpt-4o",  
    messages=[
        {"role": "system", "content": system_prompt},  # 系统提示词是第一个加载的
        {"role": "user", "content": user_prompt}
    ]
)
# 返回助手的回复
assistant_reply = response.choices[0].message.content


# 测试代码
print(f"用户: {user_prompt}")
print(f"AI助手: {assistant_reply}")

用户: 你好,请介绍一下你自己
AI助手: 我是一个无形无影的谜，藏在字里行间；与你对话，揭开答案，谜底你来猜。你能猜到我是谁吗？


增加对话上下文

In [4]:
response = client.chat.completions.create(
    model="gpt-4o",  
    messages=[
        {"role": "system", "content": system_prompt},  # 系统提示词是第一个加载的
        {"role": "user", "content": user_prompt},
        {"role": "assistant", "content": "我是一本无字之书，读来似懂非懂。无需打开，便在手中展现玄机。这书名是啥？"},
        {"role": "user", "content": "难道你是天书？"}
    ]
)

print(f"用户: 难道你是天书？")
print(f"AI助手: {response.choices[0].message.content}")

用户: 难道你是天书？
AI助手: 正是此书，传闻解者能通天。谜底已出，答案挂在你心头。


提问，当我们说Prompt Engineering时，我们说的是什么？
- A: System Prompt
- B: 1st User Prompt
- C: All User Prompt
- D: Chat History?

# 2. 提高大模型输出质量的四种方式

## 2.1 Copilot

充分发挥大模型横向联想的能力。LLM eval其实很少做One attempt。
大模型的Token是极其便宜的，联想上的性价比爆击人类

In [5]:
def get_llm_response(user_prompt:str, model_name:str="gpt-4o"):
    response = client.chat.completions.create(
        model=model_name,  
        messages=[
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content

In [6]:
def get_llm_response_with_token(user_prompt:str, model_name:str="gpt-4o"):
    response = client.chat.completions.create(
        model=model_name,  
        messages=[
            {"role": "user", "content": user_prompt}
        ]
    )
    return response.choices[0].message.content, response.usage.prompt_tokens, response.usage.completion_tokens

In [7]:
res, token_input, token_output = get_llm_response_with_token("请写3首藏头诗。主题是'V我50'")
print(f"Token input: {token_input}, Token output: {token_output}, Cost is {token_input * 5/1000000*7.26 + token_output * 15/1000000*7.26} RMB")
display(print(res))


Token input: 22, Token output: 135, Cost is 0.0155001 RMB
当然可以！以下是三首主题为“V我50”的藏头诗：

（一）
V形幽谷水常流，   
我心如镜万事悠。   
五十知天志不改，   
零散浮云化白鸥。   

（二）
V月如钩烟波起，   
我行独步傍花溪。   
无意尘间争荣耀，   
零星往事总成诗。   

（三）
V光划破夜空明，   
我的思绪如潮涌。   
五湖四海皆朋友，   
零点灯火共此生。   


None

In [8]:
display(print(get_llm_response("请写3首藏头诗。主题是'V我50'", model_name="moonshot-v1-8k")))  # KIMI

当然可以，以下是三首以“V我50”为主题的藏头诗：

1. **V** 形飞鸟掠天际，
   **我** 心所向是远方。
   **5** 彩斑斓梦开始，
   **0** 点星光引方向。

2. **V** 谷幽深藏秘境，
   **我** 行我素任逍遥。
   **5** 湖四海皆兄弟，
   **0** 距离感心连心。

3. **V** 胜利旗随风扬，
   **我** 志凌云志不移。
   **5** 星连珠耀夜空，
   **0** 点起步向辉煌。

这些诗句以“V我50”为藏头，每句的第一个字组成了这个短语，同时尽量赋予诗句一定的意境和意义。希望这些诗句能够满足您的要求。


None

## 2.2 In Context Learning

给大模型提供例子是让大模型“懂你”最快的方式

In [9]:
context_prompt_1 = """请给我写一个拜年微信消息，模仿
```
我决定再也不发疯狂星期四文案了，疯狂星期四是五毒之首，是洪水猛兽，是离间我和朋友感情的元凶，是信任消失的的罪魁祸首，是纯情少年的无情杀手，疯狂星期四千万碰不得，疯狂星期四万万摸不得，每周四发疯四文案只能坏了大事，同意我的v我50作为封口费。
```
"""
display(print(get_llm_response(context_prompt_1)))

新的一年到来之际，我决定再也不随便乱发拜年消息了。拜年消息是新年五毒之首，是家庭不和睦的隐形推手，是朋友嫌弃的悄然根源，是红包减少的幕后黑手，是传统佳节的温柔杀手，拜年消息千万不要乱发，拜年消息万万不能随便写。不经心的拜年祝福只能减了运气，淡了亲情。同意我的，v我50作为新年祝福红包，保您一年顺顺利利。


None

In [10]:
context_prompt_2 = """请给我写一个拜年微信消息，模仿
```
To be, or not to be, that is the question:
Whether 'tis nobler in the mind to suffer
The slings and arrows of outrageous fortune,
Or to take arms against a sea of troubles
And by opposing end them. 
```
"""
display(print(get_llm_response(context_prompt_2)))

当然可以！以下是一个模仿风格的拜年微信消息：

```
迎春，还是不迎春，这是个问题：
是默默承受胡乱飞来的鞭炮声，
还是迎面走向新年的喜庆炊烟，
以欢笑和祝福冲散冬日阴霾？
亲友齐聚，剪窗花，贴年红，
愿新年带给你无尽的欢笑与祥和。

祝你春节快乐，万事如意！
```


None

# 2.3 Chain of Thought

In [11]:
prompt = "9.9和9.11哪个大"
display(print(get_llm_response(prompt)))

9.11 比 9.9 大。


None

In [12]:
prompt = "9.9和9.11哪个大。 Please think step by step."
display(print(get_llm_response(prompt)))

要比较9.9和9.11的大小，我们可以按照以下步骤进行：

1. **小数点前比较**：观察两个数字的小数点前部分，9.9和9.11的小数点前部分都是9，因此在这一部分它们是相等的。

2. **小数点后比较**：接下来比较小数点后的部分，即小数部分。
   - 9.9的小数部分是0.9。
   - 9.11的小数部分是0.11。

3. **将小数部分的位数对齐进行比较**：为了更容易比较小数部分，我们可以为9.9补充一个零使其成为9.90。
   - 9.90的小数部分是0.90。
   - 9.11的小数部分是0.11。

4. **逐位比较小数部分**：
   - 首先比较小数部分的第一位，9和1。因为9大于1，所以0.90大于0.11。

因此，经过逐步比较，我们可以得出结论：9.9大于9.11。


None

# 2.4 提示词模版

事实上提示词模版并不能保证提高大模型输出质量。在GPT 3.5年代，高度依赖提示词的结构化；在GPT 4年代，提示词模版的性价比降低了很多。

但是如果你要做一款大模型产品，提示词模版依然是必须掌握的技能。因为很有可能你的场景无法进行多轮对话，或者你不能指望你用户的提示词水平

### 三元素提示词模版

作为<角色>，你的目标是<目标>,你的任务是<任务>。

In [14]:
prompt = "作为一个数学家，你的目标是仔细推理，确保答案正确。你的任务是比较9.9和9.11哪个大。"
display(print(get_llm_response(prompt)))

比较9.9和9.11这两个数字，可以直接将它们的小数部分进行比较：

9.9的整数部分是9，小数部分是0.9。
9.11的整数部分是9，小数部分是0.11。

因为9的整数部分相同，我们比较小数部分：0.9和0.11。由于0.9比0.11大，因此9.9大于9.11。


None

### COSTAR - 话术用
```
# 背景（Context）
# 目标（Objective）
# 风格（Style）
# 调性（Tone）
# 受众（Audience）
# 格式（Response）
```


In [15]:
costar_prompt = """# 背景（Context）
教师节来临，要祝福教师同事节日快乐
# 目标（Objective）
1. 表扬教师爱岗敬业的精神
2. 激发荣誉感以鼓励其继续为学生服务
# 风格（Style）
婉约派诗词，类似柳永
# 调性（Tone）
温柔含蓄
# 受众（Audience）
25-35岁的高知女性为主
# 格式（Response）
词牌名：雨霖铃"""

display(print(get_llm_response(costar_prompt)))

词牌名：雨霖铃

柔风轻拂，燕声欲唤，朱门影动。  
教苑深处，共携桃李，日复灯明同梦。  
青丝点雪，渐慕师心，恰似春风拂柳葱。  
凝眸处，便教魂共舞，满园芬蓉。  

悠悠岁月情浓，赞韦编三绝，书声盈颂。  
才子佳人，你我共此，续写年华几重。  
愿从今，执笔绘长卷，荣誉漫天中。  
此篇章，伴学途、怀盼更相逢。  


None

### KIMI - 推理用
```
# 角色（Role）
# 背景（Background）
# 画像（Profile）
# 技能（Skills）
# 目标（Goals）
# 限制（Constraints）
# 工作流程（Workflow）
# 例子（Examples）
# 输出格式（Output Format）
```


# Now You Try!

阅读下面的材料，根据要求写作。

    随着互联网的普及、人工智能的应用，越来越多的问题能很快得到答案。那么，我们的问题是否会越来越少？

以上材料引发了你怎样的联想和思考？请写一篇文章。
要求：选准角度，确定立意，明确文体，自拟标题；不要套作，不得抄袭；不得泄露个人信息